In [15]:
using SymPy
using LinearAlgebra
@vars Jhat_0 Jhat_2 l1 L1 l2 m1 m2 g b1 b2 t thetadot1 thetadot2 thetaddot1 thetaddot2 tau1 tau2 z_1 z_2 z_3 z_4 u_1 u_2
theta1 = SymFunction("theta1")(t)
theta2 = SymFunction("theta2")(t);

In [16]:
torque1 = diff(diff(theta1,t),t)*(Jhat_0+Jhat_2*(sin(theta2))^2)+diff(diff(theta2,t),t)*m2*L1*l2*cos(theta2)-(diff(theta2,t))^2*m2*L1*l2*sin(theta2)+Jhat_2*diff(theta1,t)*diff(theta2,t)*sin(2*theta2)+b1*diff(theta1,t)
cleantorque1 = torque1.subs([(diff(diff(theta1,t)),thetaddot1),(diff(diff(theta2,t)),thetaddot2),(diff(theta1,t),thetadot1),(diff(theta2,t),thetadot2)])

                                                                 2            
Ĵ₂⋅θ̇₁⋅θ̇₂⋅sin(2⋅θ₂(t)) + L₁⋅l₂⋅m₂⋅θ̈₂⋅cos(θ₂(t)) - L₁⋅l₂⋅m₂⋅θ̇₂ ⋅sin(θ₂(t)) 

               ⎛             2       ⎞
+ b₁⋅θ̇₁ + θ̈₁⋅⎝Ĵ₀ + Ĵ₂⋅sin (θ₂(t))⎠

In [17]:
torque2 = diff(diff(theta1,t),t)*m2*L1*l2*cos(theta2)+Jhat_2*diff(diff(theta2,t),t)-.5*(diff(theta2,t))^2*Jhat_2*sin(2*theta2)+b2*(diff(theta2,t))+g*m2*l2*sin(theta2)
cleantorque2 = torque2.subs([(diff(diff(theta1,t)),thetaddot1),(diff(diff(theta2,t)),thetaddot2),(diff(theta1,t),thetadot1),(diff(theta2,t),thetadot2)])

                     2                                                        
Ĵ₂⋅θ̈₂ - 0.5⋅Ĵ₂⋅θ̇₂ ⋅sin(2⋅θ₂(t)) + L₁⋅l₂⋅m₂⋅θ̈₁⋅cos(θ₂(t)) + b₂⋅θ̇₂ + g⋅l₂⋅

             
m₂⋅sin(θ₂(t))

Ok, I solved for the torques, but to use the solve function, I need an expression that is equal to zero

In [18]:
zero1 = -tau1 - torque1
zero2 = -tau2 - torque2
sol = solve( [zero1 , zero2] , [diff(theta1,t,t) , diff(theta2,t,t)])

Dict{Any,Any} with 2 entries:
  Derivative(theta1(t), (t, 2)) => (-Jhat_2*(Jhat_2*sin(2.0*theta2(t))*Derivati…
  Derivative(theta2(t), (t, 2)) => (L1*l2*m2*(Jhat_2*sin(2.0*theta2(t))*Derivat…

# That just solved for thetaddot1 and thetaddot2, but I'm not sure that the expression for thetaddot1 doesnt have thetaddot2 in it and vice versa

In [19]:
rule = Dict(  theta1=>z_1, theta2=>z_2, tau1=>u_1, tau2=>u_2 , diff(theta1,t)=>z_3, diff(theta2,t)=>z_4 )

Dict{Sym,Sym} with 6 entries:
  theta2(t)                => z_2
  tau1                     => u_1
  tau2                     => u_2
  theta1(t)                => z_1
  Derivative(theta1(t), t) => z_3
  Derivative(theta2(t), t) => z_4

In [20]:
zdot = [diff(theta1,t),diff(theta2,t), diff(theta1,t,t), diff(theta2,t,t)]

4-element Array{Sym,1}:
      Derivative(theta1(t), t)
      Derivative(theta2(t), t)
 Derivative(theta1(t), (t, 2))
 Derivative(theta2(t), (t, 2))

In [21]:
zdot = zdot.subs(sol)

4×1 Array{Sym,2}:
                                                                                                                                                                                                                                                                                                                                                                                                                                   Derivative(theta1(t), t)
                                                                                                                                                                                                                                                                                                                                                                                                                                   Derivative(theta2(t), t)
                            (-Jhat_2*(Jhat_2*sin(2.0*theta2(t))*Derivative(theta1(t), t)*Deriv

In [22]:
zdot = zdot.subs(rule)

4×1 Array{Sym,2}:
                                                                                                                                                                                                                                                                        z_3
                                                                                                                                                                                                                                                                        z_4
                      (-Jhat_2*(Jhat_2*z_3*z_4*sin(2.0*z_2) - L1*l2*m2*z_4^2*sin(z_2) + b1*z_3 + u_1) + 0.5*L1*l2*m2*(-Jhat_2*z_4^2*sin(2.0*z_2) + 2.0*b2*z_4 + 2.0*g*l2*m2*sin(z_2) + 2.0*u_2)*cos(z_2))/(Jhat_2*(Jhat_0 + Jhat_2*sin(z_2)^2) - L1^2*l2^2*m2^2*cos(z_2)^2)
 (L1*l2*m2*(Jhat_2*z_3*z_4*sin(2.0*z_2) - L1*l2*m2*z_4^2*sin(z_2) + b1*z_3 + u_1)*cos(z_2) - 0.5*(Jhat_0 + Jhat_2*sin(z_2)^2)*(-Jhat_2*z_4^2*sin(2.0*z_2) + 2.0*b2*z_4 + 2.0*g*l2*

# x = z - z_0
# r = u - u_0

# xdot = A*x + b*r

In [23]:
A1 = [ (diff( zdot[1] , z_1)) (diff( zdot[1] , z_2)) (diff( zdot[1] , z_3)) (diff( zdot[1] , z_4));
    (diff( zdot[2] , z_1)) (diff( zdot[2] , z_2)) (diff( zdot[2] , z_3)) (diff( zdot[2] , z_4));
    (diff( zdot[3] , z_1)) (diff( zdot[3] , z_2)) (diff( zdot[3] , z_3)) (diff( zdot[3] , z_4));
(diff( zdot[4] , z_1)) (diff( zdot[4] , z_2)) (diff( zdot[4] , z_3)) (diff( zdot[4] , z_4)) ]
;

In [24]:
A = [df|> subs(z_2=>pi,z_3=>0,z_4=>0)|> simplify for df in A1]

4×4 Array{Sym,2}:
 0  …                                                  0
 0                                                     1
 0     1.0*L1*b2*l2*m2/(-Jhat_0*Jhat_2 + L1^2*l2^2*m2^2)
 0       1.0*Jhat_0*b2/(-Jhat_0*Jhat_2 + L1^2*l2^2*m2^2)

In [25]:
B1 = [ (diff(zdot[1],u_1)) (diff(zdot[1],u_2));
(diff(zdot[2],u_1)) (diff(zdot[2],u_2));
(diff(zdot[3],u_1)) (diff(zdot[3],u_2));
(diff(zdot[4],u_1)) (diff(zdot[4],u_2))]

4×2 Array{Sym,2}:
                                                                                   0  …                                                                                                        0
                                                                                   0                                                                                                           0
           -Jhat_2/(Jhat_2*(Jhat_0 + Jhat_2*sin(z_2)^2) - L1^2*l2^2*m2^2*cos(z_2)^2)                     1.0*L1*l2*m2*cos(z_2)/(Jhat_2*(Jhat_0 + Jhat_2*sin(z_2)^2) - L1^2*l2^2*m2^2*cos(z_2)^2)
 L1*l2*m2*cos(z_2)/(Jhat_2*(Jhat_0 + Jhat_2*sin(z_2)^2) - L1^2*l2^2*m2^2*cos(z_2)^2)     (-1.0*Jhat_0 - 1.0*Jhat_2*sin(z_2)^2)/(Jhat_2*(Jhat_0 + Jhat_2*sin(z_2)^2) - L1^2*l2^2*m2^2*cos(z_2)^2)

In [26]:
B = [DF|> subs(z_2=>0,z_3=>0,z_4=>0)|> simplify for DF in B1]

4×2 Array{Sym,2}:
                                         0  …                                              0
                                         0                                                 0
  Jhat_2/(-Jhat_0*Jhat_2 + L1^2*l2^2*m2^2)     1.0*L1*l2*m2/(Jhat_0*Jhat_2 - L1^2*l2^2*m2^2)
 L1*l2*m2/(Jhat_0*Jhat_2 - L1^2*l2^2*m2^2)      1.0*Jhat_0/(-Jhat_0*Jhat_2 + L1^2*l2^2*m2^2)

In [27]:
sympy.julia_code(zdot)

"[z_3, z_4, (-Jhat_2.*(Jhat_2.*z_3.*z_4.*sin(2.0*z_2) - L1.*l2.*m2.*z_4.^2.*sin(z_2) + b1.*z_3 + u_1) + 0.5*L1.*l2.*m2.*(-Jhat_2.*z_4.^2.*sin(2.0*z_2) + 2.0*b2.*z_4 + 2.0*g.*l2.*m2.*sin(z_2) + 2.0*u_2).*cos(z_2))./(Jhat_2.*(Jhat_0 + Jhat_2.*sin(z_2).^2) - L1.^2.*l2.^2.*m2.^2.*cos(z_2).^2), (L1.*l2.*m2.*(Jhat_2.*z_3.*z_4.*sin(2.0*z_2) - L1.*l2.*m2.*z_4.^2.*sin(z_2) + b1.*z_3 + u_1).*cos(z_2) - 0.5*(Jhat_0 + Jhat_2.*sin(z_2).^2).*(-Jhat_2.*z_4.^2.*sin(2.0*z_2) + 2.0*b2.*z_4 + 2.0*g.*l2.*m2.*sin(z_2) + 2.0*u_2))./(Jhat_2.*(Jhat_0 + Jhat_2.*sin(z_2).^2) - L1.^2.*l2.^2.*m2.^2.*cos(z_2).^2)]"

In [33]:
sympy.latex(B) |> println

\left[\begin{matrix}0 & 0\\0 & 0\\\frac{\hat{J}_{2}}{- \hat{J}_{0} \hat{J}_{2} + L_{1}^{2} l_{2}^{2} m_{2}^{2}} & \frac{1.0 L_{1} l_{2} m_{2}}{\hat{J}_{0} \hat{J}_{2} - L_{1}^{2} l_{2}^{2} m_{2}^{2}}\\\frac{L_{1} l_{2} m_{2}}{\hat{J}_{0} \hat{J}_{2} - L_{1}^{2} l_{2}^{2} m_{2}^{2}} & \frac{1.0 \hat{J}_{0}}{- \hat{J}_{0} \hat{J}_{2} + L_{1}^{2} l_{2}^{2} m_{2}^{2}}\end{matrix}\right]


In [34]:
B

4×2 Array{Sym,2}:
                                         0  …                                              0
                                         0                                                 0
  Jhat_2/(-Jhat_0*Jhat_2 + L1^2*l2^2*m2^2)     1.0*L1*l2*m2/(Jhat_0*Jhat_2 - L1^2*l2^2*m2^2)
 L1*l2*m2/(Jhat_0*Jhat_2 - L1^2*l2^2*m2^2)      1.0*Jhat_0/(-Jhat_0*Jhat_2 + L1^2*l2^2*m2^2)